
# 🔍 venv Kernel Check (VS Code + Jupyter)

Dieses Notebook hilft dir zu verifizieren, dass dein Jupyter **Kernel** wirklich das **Python deiner venv** benutzt.

**Nutzen:**  
- Zeigt `sys.executable` (Pfad zum Python des Kernels)  
- Prüft `pip --version` und `pip list` über genau dieses Python  
- Prüft, ob ein venv-Context erkannt wird (z. B. `VIRTUAL_ENV`)  
- Gibt Hinweise, falls etwas nicht passt

> **Tipp:** Wähle in VS Code oben rechts im Notebook **Kernel auswählen** → deinen venv-Kernel (z. B. *Python (meinvenv)*).


In [ ]:

import sys, os, platform

print("Python-Version:", sys.version)
print("Executable:", sys.executable)
print("Platform:", platform.platform())
print("Arbeitsverzeichnis:", os.getcwd())

# Praktisch zum Vergleichen der Pfade
import site
try:
    print("site.getsitepackages():", site.getsitepackages())
except Exception as e:
    print("site.getsitepackages() nicht verfügbar:", e)
print("sys.path[0..3]:", sys.path[:3])


In [ ]:

import subprocess, sys

print("pip --version (über dieses Python):")
subprocess.run([sys.executable, "-m", "pip", "--version"], check=False)

print("\nAktive Pakete (Kurzliste):")
subprocess.run([sys.executable, "-m", "pip", "list"], check=False)


In [ ]:

import os, sys, pathlib

exe = pathlib.Path(sys.executable).as_posix()
venv_env = os.environ.get("VIRTUAL_ENV")

print("VIRTUAL_ENV:", venv_env)
in_venv = (getattr(sys, 'base_prefix', sys.prefix) != sys.prefix) or bool(venv_env)

print("In venv erkannt?:", in_venv)

# Heuristik: schaut, ob typische Ordnerbestandteile von venv im Pfad liegen
hints = ["venv", ".venv", "env", ".env", "Env", "pyenv"]
is_path_suspicious = not any(h in exe for h in hints)

if in_venv and is_path_suspicious:
    print("⚠️ Hinweis: Es sieht nach venv aus, aber der Python-Pfad enthält keinen klassischen venv-Ordnernamen.")
elif not in_venv:
    print("❌ Es wirkt so, als wäre dies KEINE venv. Prüfe den gewählten Kernel in VS Code.")
else:
    print("✅ venv erkannt und Pfad sieht plausibel aus.")


In [ ]:

import shutil, sys, subprocess, platform

print("shutil.which('python'):", shutil.which("python"))
cmd = ["where", "python"] if platform.system().lower().startswith("win") else ["which", "-a", "python"]
print("System-Python-Suche (nur Info):")
subprocess.run(cmd, check=False)



## ✅ Interpretation

- **`Executable`** sollte auf dein venv zeigen, z. B. `.../.venv/bin/python` (Linux/Mac) oder `...\.venv\Scripts\python.exe` (Windows).
- **`pip --version`** sollte die **gleiche** Python-Installation referenzieren.
- **`VIRTUAL_ENV`** ist oft gesetzt, wenn die Umgebung korrekt aktiv ist (nicht zwingend in jedem Setup erforderlich).
- Wenn oben **❌ KEINE venv** steht, wähle in VS Code den richtigen Kernel aus.

---

### Häufige Fixes
1. `pip install ipykernel` **im venv** installieren.
2. Kernel registrieren:
   ```bash
   python -m ipykernel install --user --name=meinvenv --display-name "Python (meinvenv)"
   ```
3. VS Code neu laden und im Notebook **Kernel → Python (meinvenv)** auswählen.
